In [ ]:
# Import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import math

In [ ]:
qs_url = 'https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt?_=1508694016501'
qs_r = requests.get(qs_url)

In [ ]:
qs_uni_dict = qs_r.json()

create the data frame

In [ ]:
qs_columns = ['Name','Region','Location','QS Rank','QS Score','No staffs', 'No internat staffs', 'No students', 'No internat students']
qs_uni_rank = pd.DataFrame(index = [], columns=qs_columns)
qs_uni_rank.head()

In [ ]:
def get_numbers_stud_staff(url):
    r_uni = requests.get("https://www.topuniversities.com" + url)
    
    html_request_uni = BeautifulSoup(r_uni.text, 'html.parser')
    
    header_staff = html_request_uni.find('div',class_='faculty-main wrapper col-md-4')
    header_stud_total = html_request_uni.find('div',class_='students-main wrapper col-md-4')
    header_stud_inter = html_request_uni.find('div',class_='int-students-main wrapper col-md-4')
    
    if ((header_staff == None) & (header_stud_total == None) & (header_stud_inter == None)):
        return (-1,-1,-1,-1)
        

    subdiv_staff_total = header_staff.find('div', class_='total faculty')
    subdiv_staff_inter = header_staff.find('div', class_= 'inter faculty')
    subdiv_stud_total = header_stud_total.find('div', class_='total student')
    subdiv_stud_inter = header_stud_inter.find('div', class_='total inter')
    
    num_staff_total = int(subdiv_staff_total.find('div', class_ = 'number').text[1:-1].replace(',',''))
    if (subdiv_staff_inter != None):
        num_staff_inter = int(subdiv_staff_inter.find('div', class_ = 'number').text[1:-1].replace(',',''))
    else:
        num_staff_inter = -1    
    num_stud_total = int(subdiv_stud_total.find('div', class_ = 'number').text[1:-1].replace(',',''))
    num_stud_inter = int(subdiv_stud_inter.find('div', class_ = 'number').text[1:-1].replace(',',''))
    
    return (num_staff_total, num_staff_inter, num_stud_total, num_stud_inter)

In [ ]:
uni_list = qs_uni_dict['data']
i = 0
for uni in uni_list[:200]:
    i+=1
    if (i%40 == 0):
        print(i)
    (tot_staff, inter_staff, tot_stud, inter_stud) = get_numbers_stud_staff(uni['url'])
    uni_df = pd.DataFrame([[uni['title'],uni['region'], uni['country'], uni['rank_display'].replace('=',' '),
                            uni['score'], tot_staff, inter_staff, tot_stud, inter_stud]], columns = qs_columns)
    qs_uni_rank = qs_uni_rank.append(uni_df)

qs_uni_rank.index = list(range(1,201))


# Second Website: THE ranking

Note that as the number students, the number of staffs and the number of international students are already mentionned in the QS ranking website they are not scrapped from the Times University Ranking website. We indeed assume that the information provided by the QS ranking website are reliable.

In [ ]:
the_r = requests.get("https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json")

the_uni_dict = the_r.json()['data'][:200]

the_columns = ['Name','Location','THE Rank','THE Score']
#the_columns = ['Name','Location','THE Rank','Score', 'No students', 'International students ratio','No students per staff']
the_uni_rank = pd.DataFrame(index = [], columns=the_columns)
the_uni_rank.head()

In [ ]:
for uni in the_uni_dict:
    uni_df = pd.DataFrame([[uni['name'],uni['location'], uni['rank'],uni['scores_overall']]],columns = the_columns)
    the_uni_rank = the_uni_rank.append(uni_df)
    
the_uni_rank.index = list(range(1,201))

In [ ]:
print(qs_columns)
print(the_columns)

###### Printing the two DataFrames

In [ ]:
qs_uni_rank.head()

In [ ]:
the_uni_rank.head()

# Merging the two DataFrames

In [ ]:
# Removing brackets and choosing the option that is less likely to be an acronym
for ind_qs, qs_uni_name in enumerate(qs_uni_rank['Name']):
    if ('(' in qs_uni_name):
        first_brack = qs_uni_name.index('(')
        brack_content = qs_uni_name[first_brack+1:-1]
        if (' ' in brack_content):
            qs_uni_rank['Name'].iloc[ind_qs] = brack_content
        else:
            qs_uni_rank['Name'].iloc[ind_qs] = qs_uni_name[:first_brack-1]
         

There is only one difference in the location definition between both DataFrames that is fixed now. Indeed it is necessary to do so for the merging operation to not fail.

In [ ]:
print("Before changing:")
print(qs_uni_rank[qs_uni_rank['Name'] == 'Lomonosov Moscow State University']['Location'])
print(the_uni_rank[the_uni_rank['Name'] == 'Lomonosov Moscow State University']['Location'])
the_uni_rank.loc[194, 'Location'] = 'Russia'
print()
print("After changing:")
print(the_uni_rank[the_uni_rank['Name'] == 'Lomonosov Moscow State University']['Location'])


In [ ]:
merged_uni_rank = pd.merge(qs_uni_rank, the_uni_rank, how = "outer")
merged_uni_rank = merged_uni_rank.set_index('Name')
merged_uni_rank.index.is_unique

In [ ]:
merged_uni_rank.head()